In [1]:
from studies.gwilliams2023 import Gwilliams2023
from studies.armeini2022 import Armeini2022


study = Gwilliams2023(
    batch_type="audiotext",
    download=False,
)

rec = study.recordings[0][0][0]
raw = rec.load_raw(load_data=True)
events = rec.load_events(raw, options="both")
word_events = events["word"]
word_events

Loading Gwilliams2023 with batch type audiotext


,onset,duration,word
0,23.506,0.30,Tara
1,23.816,0.24,stood
2,24.056,0.37,stock
3,24.586,0.40,still
4,25.136,0.41,waiting
...,...,...,...
663,361.097,0.17,end
664,361.277,0.14,for
665,361.487,0.58,project
666,362.207,0.15,and


In [ ]:
from dataloader import DataLoader

add_timestamps = True

dataloader = DataLoader(
    buffer_size=30,
    max_cache_size_gb=400,
    cache_dir="cache",
    notch_filter=True,
    frequency_bands={"all": (0.5, 80)},
    scaling="both",
    brain_clipping=None,
    baseline_window=0.5,
    new_freq=200,
    delay=0.15,
    batch_types={"audiotext": 1},
    batch_kwargs={
        "audiotext": {
            "max_random_shift": 1.0,
            "window_size": 4,
            "window_stride": 1,
            "audio_sample_rate": 16000,
            "hop_length": 160,
            "audio_processor": "openai/whisper-tiny.en",
            "add_timestamps": add_timestamps,
        }
    },
)
dataloader.start_fetching(recordings=[rec])
batch = dataloader.get_recording()

brain, audio, recording = (
    batch.brain_segments["all"], # .to(device)
    batch.audio_segments, # .to(device)
    recording
    batch.recording,
)

2025-02-28 20:08:31,422	INFO worker.py:1821 -- Started a local Ray instance.


In [ ]:
# from train.training_session_v1 import load_training_session
# import multiprocessing
# import torch

# device = "cuda"

# session = load_training_session(
#     save_path="saves/phase2/architecture/task/transformers/4C4Con_d256/epoch_39",
#     studies={"gwilliams2023": "audio"},
#     data_path="/home/ubuntu/storage-texas/data",
#     cache_name="cache",
# )

# dataloader = session.get_dataloader(buffer_size=1, num_workers=1, max_cache_size=100)

# # Unseen both
# # recording = session.studies["gwilliams2023"].recordings[19][0][0]

# # Seen
# # recording = session.studies["gwilliams2023"].recordings[15][0][1]

# # Unseen task
# # recording = session.studies["gwilliams2023"].recordings[18][0][0]

# # Unseen subject
# recording = session.studies["gwilliams2023"].recordings[19][0][1]

# print(
#     f"Showing recording: {recording.study_name}_{recording.subject_id}_{recording.task_id}"
# )

# dataloader.start_fetching(recordings=[recording])
# batch = dataloader.get_recording()
# brain, audio, recording = (
#     batch.brain_segments["all"].to(device),
#     batch.audio_segments.to(device),
#     batch.recording,
# )

# conditions = {
#     "study": f"{recording.study_name}",
#     "subject": f"{recording.study_name}_{recording.subject_id}",
# }
# session.model.to(device).eval()

# # with torch.no_grad():
# #     (
# #         x,  # [B, C, T]
# #         quantizer_metrics,
# #         channel_weights,
# #         hidden_outputs,
# #         encoder_hidden_states,  # L * [B, T, D]
# #     ) = session.model(
# #         x=[brain],
# #         recording=[recording],
# #         conditions=[conditions],
# #         mel=[audio],
# #         train=False,
# #         return_hidden_outputs=False,
# #     )

# dataloader.stop()